In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
from tabulate import tabulate

from classiq import *
from classiq.execution import (
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
    ExecutionPreferences,
)